In [1]:
import dask.array as da
from transform import DaskImage, TransformSpec
import numpy as np
import nibabel as nib

# function to extract ome zarr subvolume around nifti label from e.g. downsampled nii
in_zarr = '/cifs/prado/Kate/AK_Pipeline_LSM/spimprep/bids/sub-o28/micr/sub-o28_sample-brain_acq-prestitched_SPIM.ome.zarr/'
tiny_zarr='/local/SPIMquant/test_out/results/sub-o28/micr/sub-o28_sample-brain_acq-prestitched_desc-tiny_SPIM.ome.zarr'
in_lbl_nii='/local/SPIMquant/test_out/results/sub-o28/micr/sub-o28_sample-brain_acq-prestitched_from-ABAv3_level-5_desc-deform_dseg.nii.gz'

zarr_img = DaskImage.from_path(in_zarr) 
nii_label = DaskImage.from_path(in_lbl_nii)
print('ras2vox')
print(zarr_img.ras2vox) 
print(zarr_img)
print(nii_label.darr.shape)

print(f'affine from label nii: {nib.load(in_lbl_nii).affine}')
print(nib.load(in_lbl_nii).get_fdata()[173,148,60])

nii_label.darr.max().compute()
print(nii_label)


aba_label=457

darr_label = da.zeros_like(nii_label.darr)
darr_label[nii_label.darr==aba_label] = 1



[{'scale': [1, 0.004, 0.0018, 0.0018], 'type': 'scale'}]
[{'scale': [1, 0.004, 0.0018, 0.0018], 'type': 'scale'}]
ras2vox
TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[  -0.        ,   -0.        , -250.        ,   -0.        ],
       [  -0.        , -555.55555556,   -0.        ,   -0.        ],
       [-555.55555556,   -0.        ,   -0.        ,   -0.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None)
DaskImage(darr=dask.array<getitem, shape=(1, 2212, 11732, 7439), dtype=uint16, chunksize=(1, 1, 4096, 4096), chunktype=numpy.ndarray>, ras2vox=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[  -0.        ,   -0.        , -250.        ,   -0.        ],
       [  -0.        , -555.55555556,   -0.        ,   -0.        ],
       [-555.55555556,   -0.        ,   -0.        ,   -0.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ]]), disp_xy

In [2]:
darr_label = da.zeros_like(nii_label.darr)

darr_label[nii_label.darr==aba_label] = 1

indices = da.argwhere(darr_label).compute()
# Compute the minimum and maximum extents in each dimension
min_extent = indices.min(axis=0).reshape((4,1))
max_extent = indices.max(axis=0).reshape((4,1))

#NEED TO INTERPOLATE EXTENTS FROM VOX (DS LEVEL) -> RAS (PHYS)-> VOX (FULL RES)

#get bounds
print(min_extent)
print(max_extent)


[[  0]
 [118]
 [126]
 [ 41]]
[[  0]
 [201]
 [185]
 [191]]


In [3]:
extents_flo = np.hstack((min_extent[1:,:],max_extent[1:,:]))


#try reordered 
#extents_flo = np.flip(extents_flo,0)


print(extents_flo.shape)
extents_ref = np.round(nii_label.apply_transform_flo_to_ref_indices(TransformSpec.affine_ras_from_array(np.eye(4)),
                                    ref_dimg=zarr_img,
                                    indices=extents_flo))
print('extents flo')
print(extents_flo)

print('extents ref')
print(extents_ref)


print('fov size flo')
print(extents_flo[:,1] - extents_flo[:,0])

print('fov size ref')
print(extents_ref[:,1] - extents_ref[:,0])

(3, 2)
[[118. 201.]
 [126. 185.]
 [ 41. 191.]
 [  1.   1.]]
TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[-0.0576,  0.    ,  0.    , -0.    ],
       [ 0.    , -0.0576,  0.    , -0.    ],
       [ 0.    ,  0.    , -0.032 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None)
[[ -6.79679988 -11.5775998 ]
 [ -7.25759988 -10.65599982]
 [ -1.31200006  -6.11200029]
 [  1.           1.        ]]
TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None)
[[ -6.79679988 -11.5775998 ]
 [ -7.25759988 -10.65599982]
 [ -1.31200006  -6.11200029]
 [  1.           1.        ]]
TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[  -0.        ,   -0.        , -250.        ,   -0.        ],
       [  -0.        , -555.55555556,   -0.        ,   -0.        ],
 

In [4]:
              
zarr_img.darr = zarr_img.darr[:,
                        extents_ref[0,0]:extents_ref[0,1]+1, 
                        extents_ref[1,0]:extents_ref[1,1]+1, 
                        extents_ref[2,0]:extents_ref[2,1]+1]
print(zarr_img.darr.shape)

zarr_img.to_ome_zarr('test_subvol_img.ome.zarr')

nii_label.darr = nii_label.darr[:,
                        extents_flo[0,0]:extents_flo[0,1]+1, 
                        extents_flo[1,0]:extents_flo[1,1]+1, 
                        extents_flo[2,0]:extents_flo[2,1]+1]

#write out labelmap too
nii_label.to_ome_zarr('test_subvol_lbl.ome.zarr')


    

(1, 1201, 1889, 2657)
False
writing full-res image to zarr single threaded
[########################################] | 100% Completed | 140.66 s
writing downsampled images to zarr
[########################################] | 100% Completed | 114.80 s
True
writing full-res image to zarr single threaded
[########################################] | 100% Completed | 101.61 ms
writing downsampled images to zarr
[########################################] | 100% Completed | 102.19 ms


In [14]:
extract_subvol_ome_zarr()

[{'scale': [1, 0.004, 0.0018, 0.0018], 'type': 'scale'}]
vox2ras after reading zarr
[[-0.004   0.      0.      0.    ]
 [ 0.     -0.0018  0.      0.    ]
 [ 0.      0.     -0.0018  0.    ]
 [ 0.      0.      0.      1.    ]]
[{'scale': [1, 0.004, 0.0018, 0.0018], 'type': 'scale'}]
vox2ras after reading zarr
[[-0.004   0.      0.      0.    ]
 [ 0.     -0.0018  0.      0.    ]
 [ 0.      0.     -0.0018  0.    ]
 [ 0.      0.      0.      1.    ]]
ras2vox
TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[-250.        ,   -0.        ,   -0.        ,   -0.        ],
       [  -0.        , -555.55555556,   -0.        ,   -0.        ],
       [  -0.        ,   -0.        , -555.55555556,   -0.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None)
DaskImage(darr=dask.array<getitem, shape=(1, 2212, 11732, 7439), dtype=uint16, chunksize=(1, 1, 4096, 4096), chunktype=numpy.ndarray>, ras2vox=TransformS

In [ ]:
zarr_img = DaskImage.from_path(in_zarr) 
zarr_img.vox2ras @ np.array([0,0,0,1]).reshape((4,1))

In [3]:
zarr_img.vox2ras.affine @ np.array([1,1,1,1])

NameError: name 'zarr_img' is not defined

In [29]:
zarr_img

DaskImage(darr=dask.array<getitem, shape=(1, 2212, 11732, 7439), dtype=uint16, chunksize=(1, 1, 4096, 4096), chunktype=numpy.ndarray>, ras2vox=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[  -0.        ,   -0.        , -250.        ,   -0.        ],
       [  -0.        , -555.55555556,   -0.        ,   -0.        ],
       [-555.55555556,   -0.        ,   -0.        ,   -0.        ],
       [   0.        ,    0.        ,    0.        ,    1.        ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None), vox2ras=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[ 0.    ,  0.    , -0.0018,  0.    ],
       [ 0.    , -0.0018,  0.    ,  0.    ],
       [-0.004 ,  0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None), axes_nifti=False)

In [30]:
label_img

NameError: name 'label_img' is not defined

In [32]:
in_lbl_nii='/local/SPIMquant/test_out/results/sub-o28/micr/sub-o28_sample-brain_acq-prestitched_from-ABAv3_level-5_desc-deform_dseg.nii.gz'

daskimg_from_nii = DaskImage.from_path(in_lbl_nii)

In [34]:
daskimg_from_nii.to_ome_zarr('from_nii.ome.zarr')

[-0.0576 -0.0576 -0.032 ]
writing full-res image to zarr single threaded
[########################################] | 100% Completed | 202.37 ms
writing downsampled images to zarr
[########################################] | 100% Completed | 1.01 sms


In [35]:
daskimg_from_zarr_from_nii = DaskImage.from_path('from_nii.ome.zarr')

[{'scale': [1, -0.03200000151991844, -0.0575999990105629, -0.0575999990105629], 'type': 'scale'}]
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
[[0.0576 0.     0.     0.    ]
 [0.     0.0576 0.     0.    ]
 [0.     0.     0.032  0.    ]
 [0.     0.     0.     1.    ]]
[{'scale': [1, -0.03200000151991844, -0.0575999990105629, -0.0575999990105629], 'type': 'scale'}]
[[0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]
[[0.0576 0.     0.     0.    ]
 [0.     0.0576 0.     0.    ]
 [0.     0.     0.032  0.    ]
 [0.     0.     0.     1.    ]]


In [36]:
daskimg_from_zarr_from_nii

DaskImage(darr=dask.array<getitem, shape=(1, 277, 366, 232), dtype=float64, chunksize=(1, 277, 196, 232), chunktype=numpy.ndarray>, ras2vox=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[ 0.        ,  0.        , 31.24999852,  0.        ],
       [ 0.        , 17.36111141,  0.        ,  0.        ],
       [17.36111141,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None), vox2ras=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[0.    , 0.    , 0.0576, 0.    ],
       [0.    , 0.0576, 0.    , 0.    ],
       [0.032 , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 1.    ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None), axes_nifti=False)

In [37]:
daskimg_from_nii

DaskImage(darr=dask.array<array, shape=(1, 232, 366, 277), dtype=float64, chunksize=(1, 232, 268, 268), chunktype=numpy.ndarray>, ras2vox=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[-17.36111141,  -0.        ,  -0.        ,  -0.        ],
       [ -0.        , -17.36111141,  -0.        ,  -0.        ],
       [ -0.        ,  -0.        , -31.24999852,  -0.        ],
       [  0.        ,   0.        ,   0.        ,   1.        ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None), vox2ras=TransformSpec(tfm_type=<TransformType.AFFINE_RAS: 1>, affine=array([[-0.0576,  0.    ,  0.    , -0.    ],
       [ 0.    , -0.0576,  0.    , -0.    ],
       [ 0.    ,  0.    , -0.032 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]]), disp_xyz=None, disp_grid=None, disp_ras2vox=None), axes_nifti=True)

In [38]:
daskimg_from_zarr_from_nii.to_nifti('from_zarr_from_nii.nii')

In [39]:
import nibabel as nib
nib_to_zarr_and_back = nib.load('from_zarr_from_nii.nii')
nib_to_zarr_and_back.affine

array([[0.    , 0.    , 0.0576, 0.    ],
       [0.    , 0.0576, 0.    , 0.    ],
       [0.032 , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 1.    ]])

In [42]:
nib_orig = nib.load(in_lbl_nii)
nib_orig.affine

array([[-0.0576,  0.    ,  0.    , -0.    ],
       [ 0.    , -0.0576,  0.    , -0.    ],
       [ 0.    ,  0.    , -0.032 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [44]:
daskimg_from_nii.to_nifti('from_nii.nii')

In [45]:
nib_from_nii_to_dimg_to_nii = nib.load('from_nii.nii')
nib_from_nii_to_dimg_to_nii.affine

array([[-0.0576,  0.    ,  0.    , -0.    ],
       [ 0.    , -0.0576,  0.    , -0.    ],
       [ 0.    ,  0.    , -0.032 ,  0.    ],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [47]:
from pathlib import Path

p = Path('test.nii')
p.suffixes

['.nii']